In [19]:
import os
import numpy as np

import tensorflow as tf

In [79]:
# create a list to all tensorboard log file
# logs_directory = "../../tmp/tensorboard"
logs_directory = "../standalone/tensorboard/combination"

all_log_dir = os.listdir(logs_directory)

all_log_path = []
for log_dir in all_log_dir:
    log_dir_path = os.path.join(logs_directory, log_dir)
    all_log_path.append(os.path.join(log_dir_path, os.listdir(log_dir_path)[0]))


In [80]:
def get_log_data(path):
    logs = dict(zip(tags, [[] for _ in range(len(tags))]))

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag in tags:
                    logs[value.tag].append(value.simple_value)
    
    return logs

# Get all the logs

In [81]:
tags = ["train/acc_1", "val/acc_1"]
all_logs = dict(zip(all_log_path, [None for _ in range(len(all_log_path))]))

# for path in all_log_path:
for path in all_log_path:
    all_logs[path] = get_log_data(path)

# Calculate the stats for all the logs

In [82]:
all_logs_stats = dict(zip(all_log_path, [{} for _ in range(len(all_log_path))]))

for path in all_log_path:
    for tag in all_logs[path]:
        all_logs_stats[path][tag] = {"max": None, "min": None}
        all_logs_stats[path][tag]["max"] = np.asarray(all_logs[path][tag]).max()
        all_logs_stats[path][tag]["min"] = np.asarray(all_logs[path][tag]).min()

In [83]:
# search in the stat
def get_of(tag, key, search: str = None) -> list:
    output = []
    
    for path in all_logs_stats.keys():
        if search is not None:
            if search in path:
                output.append(all_logs_stats[path][tag][key])
        else:
            output.append(all_logs_stats[path][tag][key])
    
    return output

In [85]:
val = get_of("train/acc_1", "max")

val = np.array(val)
print("mean: ", val.mean())
print("std:  ", val.std())
print("min:  ", val.min())
print("max:  ", val.max())

mean:  0.5627240091562271
std:   0.0342539292860514
min:   0.46117085218429565
max:   0.59617680311203
